In [1]:
pip install --upgrade langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.9/922.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip install unstructured -q                    # Dependencies for loading unstructured data.
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils                  # Dependency for loading PDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.7 MB/s eta 0:00

In [3]:
!pip install tiktoken -q
!pip install pinecone-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 17.6 MB/s eta 0:00:00


## IMPORTING LIBRARIES

In [1]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## LOADING DATASET

In [2]:
directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

In [3]:
def split_documents(documents, chunk_size=1000, chunk_overlap=20):
  text_split = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  chunks = text_split.split_documents(documents)
  return chunks

chunks = split_documents(documents)
print(len(chunks))

237


In [4]:
print(chunks[5].page_content)

Baltistan, known collectively after 2009 as Gilgit-Baltistan (formerly the Northern Areas).

Relief and drainage

Pakistan is situated at the western end of the great

Indo-Gangetic Plain. Of the total area of the country,

about three-fifths consists of rough mountainous

terrain and plateaus, and the remaining two-fifths

constitutes a wide expanse of level plain. The land

Physical features of Pakistan

can be divided into five major regions: the Himalayan

and Karakoram ranges and their subranges; the Hindu

Kush and western mountains; the Balochistan plateau; the submontane plateau (Potwar

Plateau, Salt Range, trans-Indus plain, and Sialkot area); and the Indus River plain. Within

each major division there are further subdivisions, including a number of desert areas.

The Himalayan and Karakoram ranges

The Himalayas, which have long been a physical and

cultural divide between South and Central Asia, form

the northern rampart of the subcontinent, and their


In [5]:
pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=7ce4e6b66455d9111d4044e09a2be0741bbadb2690f2e2f336036787d232b884
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

query_result = embeddings.embed_query("Hello world")
len(query_result)

768

In [8]:
!pip install pinecone-client -q

In [9]:
import pinecone
#from langchain.vectorstores import pinecone

pinecone.init(
    api_key="1523033c-3666-40fc-a544-03600265eba0",
    environment="us-west4-gcp-free"
)

index_name = "text-analytics-project-embeddings"

index = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

In [10]:
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

query = "what is the capital of pakistan"
similar_docs = get_similiar_docs(query , score= True)
similar_docs

[(Document(page_content='Pakistan\n\nPakistan, populous multiethnic country of South\n\nAsia. Having a predominately Indo-Iranian speaking\n\npopulation, Pakistan has historically and culturally\n\nbeen associated with its neighbours Iran, Afghanistan,\n\nand India. Since Pakistan and India achieved\n\nPakistan\n\nindependence in 1947, Pakistan has been\n\ndistinguished from its larger southeastern neighbour\n\nby its overwhelmingly Muslim population (as opposed to the predominance of Hindus in\n\nIndia). Pakistan has struggled throughout its existence to attain political stability and\n\nsustained social development. Its capital is Islamabad, in the foothills of the Himalayas in\n\nthe northern part of the country, and its largest city is Karachi, in the south on the coast of\n\nthe Arabian Sea.\n\nPakistan was brought into being at the time of the\n\npartition of British India, in response to the demands\n\nof Islamic nationalists: as articulated by the All India\n\nMuslim League und

## APPLYING HUGGING FACE MODEL

In [11]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [14]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [15]:
from langchain.llms import HuggingFaceHub

repo_id = "bigcode/santacoder"

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":1, "max_length":94})

In [16]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer


In [17]:
query = "what is largest province of pakistan ?"
answer = get_answer(query)
print(answer)

 Punjab

Question: what is largest province of pakistan ?
Helpful Answer


## APPLYING GOOSE AI MODEL

In [18]:
import os
from langchain.llms import GooseAI
from langchain import PromptTemplate, LLMChain

In [19]:
GOOSEAI_API_KEY = getpass()

··········


In [20]:
os.environ["GOOSEAI_API_KEY"] = GOOSEAI_API_KEY

In [21]:
llm = GooseAI(model_name = 'gpt-neo-20b')

In [22]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [23]:
query = "who was mohammad ali jinnah ?"
answer = get_answer(query)
print(answer)


Mohammed Ali Jinnah was a

important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian Muslims. He was

a member of the All India Muslim League.
Mohammad Ali Jinnah was a leader of the Indian Muslims. He was

an important leader of the Indian


## APPLYING COHERE AI MODEL

In [47]:
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from langchain.llms import Cohere

In [49]:
llm = Cohere(model="base-light", cohere_api_key="rCTqOlfaNwEuTCO8ALXYryaAoBDmH8Yky6LncQnO")

In [50]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer


In [52]:
query = "what is the capital of pakistan ?"
answer = get_answer(query)
print(answer)

 The capital of Pakistan is Islamabad.

Most of Pakistan is on the Indus River, which flows

northward to the Arabian Sea. The Punjab province

is in the north and is the most populous. Baluchistan,

which has a desert climate, is in the southwest. The

Khyber Pakhtunkhwa province is in the northwest.

The South Asia region, including Pakistan, is

surrounded by the Indian Ocean on the east and the

Arabian Sea on the west.

Quetta, the capital of Baluchistan province, is

located in a high plateau at an altitude of about

2000 metres. Quetta has a hot, dry climate. It is

Pakistan: Demographic trends

at the beginning of the 21st century, Pakistan has

the world`s third largest population of Muslims after

India and Indonesia. Pakistan is also the most

populous Muslim-majority country in the world. Islam

has been the official state religion since independence

from Britain in 1947, and the government generally

tolerates other religions. Islam is the dominant religion in

the south 

## APPLYING REPLICATE MODEL

In [24]:
!pip install replicate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
# get a token: https://replicate.com/account

from getpass import getpass

REPLICATE_API_TOKEN = getpass()

··········


In [26]:
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [27]:
from langchain.llms import Replicate
from langchain import PromptTemplate, LLMChain

llm = Replicate(model="replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5")

In [28]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [29]:
query = "what is capital of pakistan ?"
answer = get_answer(query)
print(answer)

Islam
